In [ ]:
#------Importing Packages-----#
import warnings
warnings.filterwarnings('ignore')
import scanpy as sc
import anndata as ad
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy
import os
from scipy.io import mmread
from scipy.sparse import csr_matrix
import seaborn as sns
#Spatial data imports
import spatialdata_io
import spatialdata as sd
import spatialdata_plot
from spatialdata import get_extent
from spatialdata import bounding_box_query

In [ ]:
#-------Reading in AnnDatas-------#
Spatial_Prolif = sc.read_h5ad("/projects/b1217/HHA/Bulb_Spatial/20250516_HHA_Spatial_No_CC_Proliferation_Subset.h5ad")
Spatial_Prolif

In [ ]:
#--------Reading in Main Spatial Object--------#
sdata_prolif = sd.read_zarr("/projects/b1217/HHA/Spatial/Zarrs/EL_S2")

#------Filtering Data for Spots in prolif-----#
#Adding barcodes to metadata
sdata_prolif.tables["square_008um"].obs['barcode'] = sdata_prolif.tables["square_008um"].obs_names
print(sdata_prolif.tables["square_008um"].obs.head())
#Pulling anndata from object
adata = sdata_prolif.tables["square_008um"]
#Getting filter for cells in the prolif
prolif_mask = adata.obs["barcode"].isin(Spatial_Prolif.obs_names).values
#Filtering AnnData
adata_prolif = adata[prolif_mask, :].copy()
adata_prolif

#-----Matching Spot Order-----#
Spatial_prolif = Spatial_Prolif[adata_prolif.obs_names,:]
print(np.sum(Spatial_prolif.obs_names == adata_prolif.obs_names))

#------Adding metadata to subsetted anndata------#
adata_prolif.obs = adata_prolif.obs.join(Spatial_Prolif.obs, how = "left", on = "barcode", lsuffix = "L_", rsuffix = "R_")

#-----Adding new AnnData to SData Object-----#
sdata_prolif.tables["square_008um"] = adata_prolif
sdata_prolif.tables["square_008um"]

#------Filtering shapes to those in mask------#
#list of shapes found in the filtered spatial object
keeplist = sdata_prolif.tables["square_008um"].obs["location_id"].unique()
#All shapes
sh = sdata_prolif.shapes['20240612EL2_square_008um']
#Filtering for shapes in anndata
mask = sh.index.isin(keeplist)
sh_sub = sh[mask].copy()
sdata_prolif.shapes['20240612EL2_square_008um'] = sh_sub

In [ ]:
sdata_prolif.tables["square_008um"].obs

In [ ]:
#------Plotting TACCO Predicted CellTypes-----#
#Setting Palette and Plotting Order
ProlifPal = ['#B22222', "#6495ED"]
prolif_groups = ["Proliferating Cortex/Cuticle", "Proliferating IRS"]

#------Plotting on Full Image-------#
fig,ax=plt.subplots(figsize=(50, 50))
image = '20240612EL2_full_image'
shapes = '20240612EL2_square_008um'
#Plotting Clusters on Full Image
sdata_prolif.pl.render_images(image).pl.render_shapes(
  shapes, 
  color="ProliferativeGrouping",
  palette = ProlifPal,
    groups = prolif_groups,
  method="matplotlib").pl.show(coordinate_systems="global", title=f"bin_size=8µm", ax=ax)
#Saving as png
fig.savefig("/projects/b1217/HHA/Bulb_Seurat_Plots/Spatial/TACCO_Mapping_Spatial_ProliferativeGroups.png",
            dpi = 300, pad_inches = 0.5, bbox_inches = "tight")